In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pylab as plt
from matplotlib.pylab import rcParams
import pmdarima as pm
from pmdarima import auto_arima,arima
import warnings
from sklearn.preprocessing import MinMaxScaler
# get functions from utils.py
from utils import *
from joblib import dump
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Bidirectional, LSTM, Dense, Dropout, LayerNormalization, LeakyReLU
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

In [ ]:
ari = pd.read_csv("data_ari.csv",sep=",",dtype={'location':str,'year_week':str,
                                                'value':np.float32,'relative_humidity_2m':np.float64,
                                                'temperature_2m_max':np.float64,'temperature_2m_min':np.float64},
                                                parse_dates=['truth_date'])


In [ ]:
ili = pd.read_csv("data_ili.csv",sep=",",dtype={'location':str,'year_week':str,
                                                'value':np.float32,'relative_humidity_2m':np.float64,
                                                'temperature_2m_max':np.float64,'temperature_2m_min':np.float64},
                                                parse_dates=['truth_date'])
ili = ili.drop(columns=['Unnamed: 0']).reset_index(drop=True)

In [ ]:
def train_test_rnn(full_df, location, date, n_input=16, n_output=4):
    # Filter country and set data
    data_location = full_df[full_df['location'] == location].copy()
    data_location['truth_date'] = pd.to_datetime(data_location['truth_date'])
    data_location = data_location.set_index('truth_date')

    # features
    data = create_features(data_location)
    data.drop(columns=['location'], inplace=True)

    train = data[data.index <= date].copy()
    test = data[data.index > date].copy()

    # Scaling 
    var = ['relative_humidity_2m', 'temperature_2m_max', 'temperature_2m_min',
           'lag_value_1', 'lag_humidity_1', 'lag_temp_max_1', 'lag_temp_min_1',
           'lag_value_2', 'lag_humidity_2', 'lag_temp_max_2', 'lag_temp_min_2',
           'lag_value_3', 'lag_humidity_3', 'lag_temp_max_3', 'lag_temp_min_3',
           'lag_value_4', 'lag_humidity_4', 'lag_temp_max_4', 'lag_temp_min_4']
    
    var2 = ['value', 'week_mas_1', 'week_mas_2', 'week_mas_3']
    print(train.corr()['value'].sort_values(ascending=False))
    scal = MinMaxScaler()
    scal_2 = MinMaxScaler()

    train[var] = scal.fit_transform(train[var])
    test[var] = scal.transform(test[var])

    train[var2] = scal_2.fit_transform(train[var2])
    test[var2] = scal_2.transform(test[var2])

    # union data to generate sequences
    full_scaled = pd.concat([train, test])
    full_scaled.drop(columns=['week_mas_1', 'week_mas_2', 'week_mas_3'])
    print(full_scaled.columns)
    X_all, y_all = generate_sequences(full_scaled, target_col='value', n_input=n_input, n_output=n_output)

    # dates
    dates = full_scaled.index[n_input + n_output - 1:]

    idx = dates.get_indexer([pd.to_datetime(date)])
    
    split_idx = idx[0]

    # Train test split to train model
    X_train, y_train = X_all[:split_idx], y_all[:split_idx]
    X_test, y_test = X_all[split_idx:], y_all[split_idx:]

    return X_train, y_train, X_test, y_test, scal_2, scal, dates


In [ ]:
def create_features_rnn(data):
    """
    Create additional features for the non sequential dataset.
    """
    data = data.copy()

    # Extract year, month, day, weekday, and week from 'truth_date'
    data['year'] = data.index.year
    data['month'] = data.index.month

    week = data['year_week'].str.split('-W').str[1]
    data['week'] = week.astype(int)
    for h in range(1,53):
        data[f'lag_value_{h}'] = data['value'].shift(h)
        data[f'lag_humidity_{h}'] = data['relative_humidity_2m'].shift(h)
        data[f'lag_temp_max_{h}'] = data['temperature_2m_max'].shift(h)
        data[f'lag_temp_min_{h}'] = data['temperature_2m_min'].shift(h)
    data = data.dropna()
    # Convert cyclical categorical variables to category type
    data['month_sin'] = np.sin(2 * np.pi * data['month']/12)
    data['month_cos'] = np.cos(2 * np.pi * data['month']/12)
    data['week_sin'] = np.sin(2 * np.pi * data['week']/52)
    data['week_cos'] = np.cos(2 * np.pi * data['week']/52)
    data = data.drop(columns=['month', 'week','year_week'])
    data['week_mas_1'] = data['value'].shift(-1)
    data['week_mas_2'] = data['value'].shift(-2)
    data['week_mas_3'] = data['value'].shift(-3)
    return data

In [ ]:
mape_ari = pd.DataFrame(columns=['location','model','prediction_window','mae','rmse'])
mape_ili = pd.DataFrame(columns=['location','model','prediction_window','mae','rmse'])

In [ ]:
def build_lstm_model(X_train, y_train, n_features=76, n_input=52, epochs=100, batch_size=32,
                     n_output=4, model_name=None, country=None):

    model = Sequential([
        Bidirectional(LSTM(128, return_sequences=True, recurrent_dropout=0.2), input_shape=(n_input, n_features)),
        LayerNormalization(),
        Dropout(0.2),

        Bidirectional(LSTM(64, return_sequences=True, recurrent_dropout=0.2)),
        LayerNormalization(),
        Dropout(0.1),

        Bidirectional(LSTM(32, return_sequences=False, recurrent_dropout=0.2)),
        LayerNormalization(),
        Dropout(0.2),

        Dense(64),
        LeakyReLU(alpha=0.1),
        Dropout(0.1),

        Dense(32),
        LeakyReLU(alpha=0.1),
        Dropout(0.2),

        Dense(n_output)  
    ])

    model.compile(optimizer='adam', loss='mse')

    # Callbacks
    early_stop = EarlyStopping(monitor='val_loss', patience=30, restore_best_weights=True)
    lr_schedule = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, verbose=1)

    model.fit(X_train, y_train,
              validation_split=0.1,
              epochs=epochs,
              batch_size=batch_size,
              callbacks=[early_stop, lr_schedule],
              verbose=1)

    # Save model
    dump(model, f"models_rnn/rnn_model_{country}_{model_name}.joblib")

    return model


In [ ]:
def prediction_rnn(model,X_test,y_test,n_features=76, n_input=16, n_output=4, model_name=None, country=None, mape=None, scal_2=None,dates=None):

    y_test_df = pd.DataFrame(y_test, columns=['value', 'week_mas_1', 'week_mas_2', 'week_mas_3'])
    true_real = pd.DataFrame(scal_2.inverse_transform(y_test_df), columns=['value', 'week_mas_1', 'week_mas_2', 'week_mas_3'])
    # Rolling predictions
    rolling_raw = []
    for i in range(len(X_test)):
        input_seq = X_test[i].reshape(1, n_input, n_features)
        pred_scaled = model.predict(input_seq, verbose=0)
        rolling_raw.append(pred_scaled.flatten())

    rolling_df = pd.DataFrame(rolling_raw, columns=[f"prediction_{i+1}_weeks" for i in range(n_output)])
    rolling_df_inverse = pd.DataFrame(scal_2.inverse_transform(rolling_df), columns=[f"prediction_{i+1}_weeks" for i in range(n_output)])

    results = []
    for i in range(n_output):
        mae, rmse = eval_metrics(true_real.iloc[:, i], rolling_df_inverse.iloc[:, i])
        results.append([country, model_name, f"{i+1}_week", mae, rmse])

    mape = pd.concat([
        mape,
        pd.DataFrame(results, columns=['location', 'model', 'prediction_window', 'mae', 'rmse'])
    ], ignore_index=True)
    rolling_df_inverse.index = dates[-rolling_df_inverse.shape[0]:]
    true_real.index = dates[-true_real.shape[0]:] 
    return rolling_df_inverse, mape, true_real

In [ ]:
HU = ari[ari['location']=='HU']
HU.head(33)

In [ ]:
name_ari = ari.location.unique()
name_ili = ili.location.unique()

In [ ]:
for i in ['HU']:
    print(f"Training model for {i}")
    X_train, y_train, X_test, y_test, scal_2, scal, dates = train_test_rnn(ari,i,'2023-05-21', n_input=52, n_output=4)
    print(X_train.shape)
    model= build_lstm_model(X_train,y_train ,n_features=28, n_input=52, epochs=200, batch_size=32,
                 n_output=4, model_name='ARI', country=i)
    y_train_df = pd.DataFrame(y_train, columns=['value', 'week_mas_1', 'week_mas_2', 'week_mas_3'])
    y_train_df_inv = pd.DataFrame(scal_2.inverse_transform(y_train_df), columns=['value', 'week_mas_1', 'week_mas_2', 'week_mas_3'])
    y_train_df_inv.index = dates[:y_train_df_inv.shape[0]]
    test_aux, mape_ari, true_real= prediction_rnn(model,X_test,y_test,n_features=28, n_input=52, n_output=4, model_name='ARI', country=i, mape=mape_ari, scal_2=scal_2,dates=dates)
    test_aux['value'] = true_real['value']
    test_aux.to_csv(f'results_RNN_{i}_ari.csv',index=False)
    plot_train_test(y_train_df_inv, test_aux, 'ARI', i,'rnn')

    

In [ ]:
mape_ari

In [ ]:
y_train_df_inv

In [ ]:
test_aux

In [ ]:
mape_ari.to_csv('mape_ari_rnn.csv', index=False)

In [ ]:
for i in name_ili:
    print(f"Training model for {i}")
    X_train, y_train, X_test, y_test, scal_2, scal, dates = train_test_rnn(ili,i,'2023-10-15', n_input=52, n_output=4)
    model= build_lstm_model(X_train,y_train ,n_features=28, n_input=52, epochs=200, batch_size=32,
                 n_output=4, model_name='ILI', country=i)
    y_train_df = pd.DataFrame(y_train, columns=['value', 'week_mas_1', 'week_mas_2', 'week_mas_3'])
    y_train_df_inv = pd.DataFrame(scal_2.inverse_transform(y_train_df), columns=['value', 'week_mas_1', 'week_mas_2', 'week_mas_3'])
    y_train_df_inv.index = dates[:y_train_df_inv.shape[0]]
    test_aux, mape_ili, true_real= prediction_rnn(model,X_test,y_test,n_features=28, n_input=52, n_output=4, model_name='ILI', country=i, mape=mape_ili, scal_2=scal_2,dates=dates)
    test_aux['value'] = true_real['value']
    test_aux.to_csv(f'results_RNN_{i}_ili.csv',index=False)
    plot_train_test(y_train_df_inv, test_aux, 'ILI', i,'rnn')

In [ ]:
mape_ili.tail(30)

In [ ]:
mape_ili.to_csv('mape_ili_rnn.csv', index=False,decimal=',',sep=';', float_format='%.4f')